<a href="https://colab.research.google.com/github/KseniaLapshina/hse22_hw2/blob/main/src/hse22_hw2_bonus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Скачивание файлов близкородственной бактерии

In [1]:
!sh -c "$(curl -fsSL ftp://ftp.ncbi.nlm.nih.gov/entrez/entrezdirect/install-edirect.sh)"


Entrez Direct has been successfully downloaded and installed.

In order to complete the configuration process, please execute the following:

  echo "export PATH=\${PATH}:/root/edirect" >> ${HOME}/.bashrc

or manually edit the PATH variable assignment in your .bashrc file.

Would you like to do that automatically now? [y/N]
y
OK, done.

To activate EDirect for this terminal session, please execute the following:

export PATH=${PATH}:${HOME}/edirect



In [2]:
!$HOME/edirect/efetch -db nuccore -id HF680312 -format gb  >  T_oleivorans_MIL_1.gbk

Установка программ

In [3]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 20.1 MB/s 


In [4]:
!apt-get update

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,554 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [1,208 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [3,035 kB]
Hit:12 http://ppa.launchpad.net/cran/libg

In [5]:
!apt-get install ncbi-blast+

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  ncbi-data
The following NEW packages will be installed:
  ncbi-blast+ ncbi-data
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 13.1 MB of archives.
After this operation, 66.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 ncbi-data all 6.1.20170106-2 [3,645 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 ncbi-blast+ amd64 2.6.0-1 [9,446 kB]
Fetched 13.1 MB in 1s (11.8 MB/s)
Selecting previously unselected package ncbi-data.
(Reading database ... 123941 files and directories currently installed.)
Preparing to unpack .../ncbi-data_6.1.20170106-2_all.deb ...
Unpacking ncbi-data (6.1.20170106-2) ...
Selecting

Предсказание рибосомальной рнк

In [18]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation
import re

In [7]:
records = [rec for rec in SeqIO.read("T_oleivorans_MIL_1.gbk", "genbank").features]
records[1]

SeqFeature(FeatureLocation(ExactPosition(34), ExactPosition(1624), strand=1), type='gene')

In [8]:
posstrgenes = []
negstrgenes = []

for r in records:
    if r.type == 'rRNA':
        if r.location.strand == 1:
            posstrgenes.append([r.location.start, r.location.end])
        else:
            negstrgenes.append([r.location.start, r.location.end])

In [9]:
len(posstrgenes)

3

In [10]:
len(negstrgenes)

9

In [11]:
!$HOME/edirect/efetch -db nuccore -id HF680312 -format gene_fasta  >  T_oleivorans_MIL_1.genes.fasta

In [12]:
genome = ''
for seq in SeqIO.parse("T_oleivorans_MIL_1.genes.fasta", "fasta"):
    genome += str(seq.seq)

In [13]:
genome

'...'

In [20]:
file = open('rRNA.fasta', 'w')
for ps in posstrgenes:
    file.write(f'> rRNA {ps[0]} {ps[1]}\n{genome[ps[0]:ps[1]]}\n')
negstrgenome = Seq(genome).reverse_complement()
for ns in negstrgenes:
    file.write(f'> rRNA {ns[0]} {ns[1]}\n{negstrgenome[ns[0]:ns[1]]}\n')
file.close()

In [22]:
!wget https://raw.githubusercontent.com/KseniaLapshina/hse22_hw1/main/data/scaffolds.fa

--2022-10-19 20:10:55--  https://raw.githubusercontent.com/KseniaLapshina/hse22_hw1/main/data/scaffolds.fa
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3972818 (3.8M) [text/plain]
Saving to: ‘scaffolds.fa’

scaffolds.fa        100%[===================>]   3.79M  --.-KB/s    in 0.01s   

2022-10-19 20:10:56 (256 MB/s) - ‘scaffolds.fa’ saved [3972818/3972818]



In [23]:
!blastn -query rRNA.fasta -subject scaffolds.fa > blastn.gbk